# YouTube Ranking Data 수집해서 시각화하기

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [5]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)     # 반복적으로 사이트 열고 닫을때 error 발생

In [7]:
trs = driver.find_elements_by_tag_name('tr')
len(trs)

102

In [8]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [9]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [14]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [15]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [18]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [20]:
channels = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category, name, subscriber, view, video])

In [21]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '조회수', '영상수'])
df.head()

,카테고리,채널명,구독자수,조회수,영상수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [22]:
df.tail()

,카테고리,채널명,구독자수,조회수,영상수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,애완/반려동물,크림히어로즈,381만,13억7401만,939


- 숫자 단위(만, 억) 숫자로 바꿔주는 함수

In [26]:
def convert_unit(s):
    s = ''.join(s.split('억'))
    s = s.replace('만', '0000')
    return s

In [27]:
convert_unit('10억6557만')

'1065570000'